# Jupyter Notebook in Vscode

## Start Server

In [3]:
from deephaven_server import Server

if Server.instance is None:
    s = Server(
        port=8080, 
        jvm_args=['-Ddeephaven.dataDir=./data']
        )
    s.start()

## Log Table

In [4]:
import re
from deephaven import dtypes as dht, new_table
from deephaven.column import datetime_col, string_col
from deephaven.time import to_datetime
from deephaven_ipywidgets import DeephavenWidget

log_file_path = 'data/logs/2023-03-21-180652_support_logs/console.txt'

columns = {
    'key': [],
    'ts': [],
    'type': [],
    'msg': [],
    'body': [],
}

def get_key(type, msg):
    msg = re.sub(r"[A-Za-z0-9]{8}-([A-Za-z0-9]{4}-){3}[A-Za-z0-9]{12}", '<GUID>', msg)
    msg = re.sub(r'(.*"grpc-status-details-bin":\[")([^"]+)("\].*)', r'\1<TOKEN>\3', msg)
    msg = re.sub(r'("date":\[")([^"]+)("\])', r'\1<DATE>\3', msg)
    return type + '\t' + msg

with open(log_file_path, 'r', encoding="utf-8") as fh:
    for line in fh:
        match = re.search(r'^(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}:\d{2}.\d{3}Z) ([A-Z])\t(.*)', line)

        if match:
            ts, type, msg = match.groups()
            key = get_key(type, msg)
            # print(key)
            columns['key'].append(key)
            columns['ts'].append(to_datetime(ts))
            columns['type'].append(type)
            columns['msg'].append(msg)
            columns['body'].append('')
        else:
            columns['body'][-1] += '\n' + line

col_defs = {
    'key': dht.string,
    'ts': dht.DateTime,
    'type': dht.string,
    'msg': dht.string,
    'body': dht.string
}

log = new_table([
    datetime_col('ts', columns['ts']),
    string_col('type', columns['type']),
    string_col('msg', columns['msg']),
    string_col('body', columns['body']),
    string_col('key', columns['key']),
])

d_log = DeephavenWidget(log)
display(d_log)

DeephavenWidget(height=600, iframe_url='http://localhost:8080/iframe/table/?name=t_2db0b774_c215_4131_932f_f86…

## Counts Table

In [5]:
from deephaven import agg

counts = log.agg_by([agg.count_(col='Number')], by=['key']).sort(order_by=['key'])

d_counts = DeephavenWidget(counts)
display(d_counts)

DeephavenWidget(height=600, iframe_url='http://localhost:8080/iframe/table/?name=t_53bf2165_890d_4494_842a_766…